### XGBoost tutorial from Kaggle re-adapted

Source: https://www.kaggle.com/code/robikscube/tutorial-time-series-forecasting-with-xgboost/notebook

Source 2: https://cprosenjit.medium.com/10-time-series-forecasting-methods-we-should-know-291037d2e285

## Data

In [ ]:
import os
import modal
import hopsworks
import pandas as pd
import xgboost as xgb
import joblib
import seaborn as sns
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [ ]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
# fs is a reference to the Hopsworks Feature Store
fs = project.get_feature_store()

# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
try: 
    ###################################
    # COMBINE FEATURE VIEWS WITH JOIN #
    ###################################
    feature_view = fs.get_feature_view(name="snow_weather_data", version=1)
except:
    snow_weather_fg = fs.get_feature_group(name="snow_weather_data", version=1)
    query = snow_weather_fg.select_all()
    feature_view = fs.create_feature_view(name="snow_weather_data",
                                      version=1,
                                      description="Predicting snow depth.",
                                      labels=["hs"],
                                      query=query) 

## Train test split

In [ ]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.1)
print("X train: ", X_train, "Y train: ", y_train)

## Model XGBoost

In [ ]:
# Train our model with the Scikit-learn XGBoost algorithm using our features (X_train) and labels (y_train)
model = xgb.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=50,
    verbose=False) # Change verbose to True if you want to see it train

## Prediction on test

In [ ]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)

In [ ]:
# Compare predictions (y_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'titanic_model' directory will be saved to the model registry. Create the dir, first.
model_dir="titanic_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/titanic_model.pkl")
fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
titanic_model = mr.python.create_model(
    name="titanic_modal", 
    metrics={"accuracy" : metrics['accuracy']},
    model_schema=model_schema,
    description="Titanic survival Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
titanic_model.save(model_dir)
    
if __name__ == "__main__":
    if LOCAL == True :
        g()
    else:
        with stub.run():
            f()